In [26]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
import torch

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
torch.cuda.current_device()
torch.cuda.device(0)


In [31]:
# coding=utf-8
import argparse
import logging
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import random

import numpy as np
import pandas as pd
import torch
from transformers import (BertConfig, BertForTokenClassification,
                                  BertTokenizer)
from torch.utils.data import DataLoader

from datasets import load_datasets_and_vocabs
from model import (Aspect_Text_GAT_ours,
                    Pure_Bert, Aspect_Bert_GAT, Aspect_Text_GAT_only)
from trainer import train

logger = logging.getLogger(__name__)

In [32]:
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

def parse_args(args):
    parser = argparse.ArgumentParser()

    # Required parameters
    parser.add_argument('--dataset_name', type=str, default='rest',
                        choices=['rest', 'laptop', 'twitter'],
                        help='Choose absa dataset.')
    parser.add_argument('--output_dir', type=str, default='/data1/SHENWZH/ABSA_online/data/output-gcn',
                        help='Directory to store intermedia data, such as vocab, embeddings, tags_vocab.')
    parser.add_argument('--num_classes', type=int, default=3,
                        help='Number of classes of ABSA.')


    parser.add_argument('--cuda_id', type=str, default='3',
                        help='Choose which GPUs to run')
    parser.add_argument('--seed', type=int, default=2019,
                        help='random seed for initialization')

    # Model parameters
    parser.add_argument('--glove_dir', type=str, default='/data1/SHENWZH/wordvec',
                        help='Directory storing glove embeddings')
    parser.add_argument('--bert_model_dir', type=str, default='/data1/SHENWZH/models/bert_base',
                        help='Path to pre-trained Bert model.')
    parser.add_argument('--pure_bert', action='store_true',
                        help='Cat text and aspect, [cls] to predict.')
    parser.add_argument('--gat_bert', action='store_true',
                        help='Cat text and aspect, [cls] to predict.')

    parser.add_argument('--highway', action='store_true',
                        help='Use highway embed.')

    parser.add_argument('--num_layers', type=int, default=2,
                        help='Number of layers of bilstm or highway or elmo.')


    parser.add_argument('--add_non_connect',  type= bool, default=True,
                        help='Add a sepcial "non-connect" relation for aspect with no direct connection.')
    parser.add_argument('--multi_hop',  type= bool, default=True,
                        help='Multi hop non connection.')
    parser.add_argument('--max_hop', type = int, default=4,
                        help='max number of hops')


    parser.add_argument('--num_heads', type=int, default=6,
                        help='Number of heads for gat.')
    
    parser.add_argument('--dropout', type=float, default=0,
                        help='Dropout rate for embedding.')


    parser.add_argument('--num_gcn_layers', type=int, default=1,
                        help='Number of GCN layers.')
    parser.add_argument('--gcn_mem_dim', type=int, default=300,
                        help='Dimension of the W in GCN.')
    parser.add_argument('--gcn_dropout', type=float, default=0.2,
                        help='Dropout rate for GCN.')
    # GAT
    parser.add_argument('--gat', action='store_true',
                        help='GAT')
    parser.add_argument('--gat_our', action='store_true',
                        help='GAT_our')
    parser.add_argument('--gat_attention_type', type = str, choices=['linear','dotprod','gcn'], default='dotprod',
                        help='The attention used for gat')

    parser.add_argument('--embedding_type', type=str,default='glove', choices=['glove','bert'])
    parser.add_argument('--embedding_dim', type=int, default=300,
                        help='Dimension of glove embeddings')
    parser.add_argument('--dep_relation_embed_dim', type=int, default=300,
                        help='Dimension for dependency relation embeddings.')

    parser.add_argument('--hidden_size', type=int, default=300,
                        help='Hidden size of bilstm, in early stage.')
    parser.add_argument('--final_hidden_size', type=int, default=300,
                        help='Hidden size of bilstm, in early stage.')
    parser.add_argument('--num_mlps', type=int, default=2,
                        help='Number of mlps in the last of model.')

    # Training parameters
    parser.add_argument("--per_gpu_train_batch_size", default=16, type=int,
                        help="Batch size per GPU/CPU for training.")
    parser.add_argument("--per_gpu_eval_batch_size", default=32, type=int,
                        help="Batch size per GPU/CPU for evaluation.")
    parser.add_argument('--gradient_accumulation_steps', type=int, default=2,
                        help="Number of updates steps to accumulate before performing a backward/update pass.")
    parser.add_argument("--learning_rate", default=1e-3, type=float,
                        help="The initial learning rate for Adam.")
    
    parser.add_argument("--weight_decay", default=0.0, type=float,
                        help="Weight deay if we apply some.")
    parser.add_argument("--adam_epsilon", default=1e-8, type=float,
                        help="Epsilon for Adam optimizer.")

    parser.add_argument("--max_grad_norm", default=1.0, type=float,
                        help="Max gradient norm.")
    parser.add_argument("--num_train_epochs", default=30.0, type=float,
                        help="Total number of training epochs to perform.")
    parser.add_argument("--max_steps", default=-1, type=int,
                        help="If > 0: set total number of training steps(that update the weights) to perform. Override num_train_epochs.")
    parser.add_argument('--logging_steps', type=int, default=50,
                        help="Log every X updates steps.")
    
    return parser.parse_args(args)


def check_args(args):
    '''
    eliminate confilct situations
    
    '''
    logger.info(vars(args))

In [33]:
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt='%m/%d/%Y %H:%M:%S',
                        level=logging.INFO)
    
# Parse args
args = parse_args(['--gat_our', '--highway', '--num_heads', '7', '--dropout', '0.8', '--output_dir',
                   'output/r-gat', '--glove_dir', 'glove', '--cuda_id', '0'])
check_args(args)

# Setup CUDA, GPU training
os.environ["CUDA_VISIBLE_DEVICES"] = args.cuda_id
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args.device = device
logger.info('Device is %s', args.device)

# Set seed
set_seed(args)

# Bert, load pretrained model and tokenizer, check if neccesary to put bert here
if args.embedding_type == 'bert':
    tokenizer = BertTokenizer.from_pretrained(args.bert_model_dir)
    args.tokenizer = tokenizer

# Load datasets and vocabs
train_dataset, test_dataset, word_vocab, dep_tag_vocab, pos_tag_vocab= load_datasets_and_vocabs(args)

# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


07/07/2021 07:02:51 - INFO - __main__ -   {'dataset_name': 'rest', 'output_dir': 'output/r-gat', 'num_classes': 3, 'cuda_id': '0', 'seed': 2019, 'glove_dir': 'glove', 'bert_model_dir': '/data1/SHENWZH/models/bert_base', 'pure_bert': False, 'gat_bert': False, 'highway': True, 'num_layers': 2, 'add_non_connect': True, 'multi_hop': True, 'max_hop': 4, 'num_heads': 7, 'dropout': 0.8, 'num_gcn_layers': 1, 'gcn_mem_dim': 300, 'gcn_dropout': 0.2, 'gat': False, 'gat_our': True, 'gat_attention_type': 'dotprod', 'embedding_type': 'glove', 'embedding_dim': 300, 'dep_relation_embed_dim': 300, 'hidden_size': 300, 'final_hidden_size': 300, 'num_mlps': 2, 'per_gpu_train_batch_size': 16, 'per_gpu_eval_batch_size': 32, 'gradient_accumulation_steps': 2, 'learning_rate': 0.001, 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'num_train_epochs': 30.0, 'max_steps': -1, 'logging_steps': 50}
07/07/2021 07:02:51 - INFO - __main__ -   Device is cuda
07/07/2021 07:02:51 - INFO - datasets -   #

Aspect_Text_GAT_ours(
  (embed): Embedding(3508, 300)
  (dropout): Dropout(p=0.8, inplace=False)
  (tanh): Tanh()
  (highway_dep): Highway(
    (linear): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): Linear(in_features=300, out_features=300, bias=True)
    )
    (gate): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): Linear(in_features=300, out_features=300, bias=True)
    )
  )
  (highway): Highway(
    (linear): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): Linear(in_features=300, out_features=300, bias=True)
    )
    (gate): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): Linear(in_features=300, out_features=300, bias=True)
    )
  )
  (bilstm): LSTM(300, 300, num_layers=2, batch_first=True, bidirectional=True)
  (dep_embed): Embedding(46, 300)
  (fcs): Sequential(
    (0): Linear(in_features=1200, out_features=300, bias=Tru

In [34]:
model

Aspect_Text_GAT_ours(
  (embed): Embedding(3508, 300)
  (dropout): Dropout(p=0.8, inplace=False)
  (tanh): Tanh()
  (highway_dep): Highway(
    (linear): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): Linear(in_features=300, out_features=300, bias=True)
    )
    (gate): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): Linear(in_features=300, out_features=300, bias=True)
    )
  )
  (highway): Highway(
    (linear): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): Linear(in_features=300, out_features=300, bias=True)
    )
    (gate): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): Linear(in_features=300, out_features=300, bias=True)
    )
  )
  (bilstm): LSTM(300, 300, num_layers=2, batch_first=True, bidirectional=True)
  (dep_embed): Embedding(46, 300)
  (fcs): Sequential(
    (0): Linear(in_features=1200, out_features=300, bias=Tru

In [35]:
# Train
_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    for key in sorted(best_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_eval_result[key]))

07/07/2021 07:04:56 - INFO - trainer -   ***** Running training *****
07/07/2021 07:04:56 - INFO - trainer -     Num examples = 3602
07/07/2021 07:04:56 - INFO - trainer -     Num Epochs = 30
07/07/2021 07:04:56 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/07/2021 07:04:56 - INFO - trainer -     Gradient Accumulation steps = 2
07/07/2021 07:04:56 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
07/07/2021 07:04:59 - INFO - trainer -   ***** Running evaluation *****
07/07/2021 07:04:59 - INFO - trainer -     Num examples = 1120
07/07/2021 07:04:59 - INFO - trainer -     Batch size = 32
07/07/2021 07:05:00 - INFO - trainer -   ***** Eval results *****
07/07/20

07/07/2021 07:05:57 - INFO - trainer -     Batch size = 32
07/07/2021 07:05:58 - INFO - trainer -   ***** Eval results *****
07/07/2021 07:05:58 - INFO - trainer -     eval loss: 0.5531125336885452
07/07/2021 07:05:58 - INFO - trainer -     acc = 0.7830357142857143
07/07/2021 07:05:58 - INFO - trainer -     f1 = 0.6531542856177347
07/07/2021 07:06:01 - INFO - trainer -   ***** Running evaluation *****
07/07/2021 07:06:01 - INFO - trainer -     Num examples = 1120
07/07/2021 07:06:01 - INFO - trainer -     Batch size = 32
07/07/2021 07:06:02 - INFO - trainer -   ***** Eval results *****
07/07/2021 07:06:02 - INFO - trainer -     eval loss: 0.5123141582523073
07/07/2021 07:06:02 - INFO - trainer -     acc = 0.8098214285714286
07/07/2021 07:06:02 - INFO - trainer -     f1 = 0.7146273947186691
07/07/2021 07:06:05 - INFO - trainer -   ***** Running evaluation *****
07/07/2021 07:06:05 - INFO - trainer -     Num examples = 1120
07/07/2021 07:06:05 - INFO - trainer -     Batch size = 32
07/07

07/07/2021 07:07:03 - INFO - trainer -     Num examples = 1120
07/07/2021 07:07:03 - INFO - trainer -     Batch size = 32
07/07/2021 07:07:04 - INFO - trainer -   ***** Eval results *****
07/07/2021 07:07:04 - INFO - trainer -     eval loss: 0.5151275683726583
07/07/2021 07:07:04 - INFO - trainer -     acc = 0.7964285714285714
07/07/2021 07:07:04 - INFO - trainer -     f1 = 0.7051286726617215
Epoch:  50%|█████     | 15/30 [02:11<02:10,  8.68s/it]07/07/2021 07:07:07 - INFO - trainer -   ***** Running evaluation *****
07/07/2021 07:07:07 - INFO - trainer -     Num examples = 1120
07/07/2021 07:07:07 - INFO - trainer -     Batch size = 32
07/07/2021 07:07:08 - INFO - trainer -   ***** Eval results *****
07/07/2021 07:07:08 - INFO - trainer -     eval loss: 0.5232623447264944
07/07/2021 07:07:08 - INFO - trainer -     acc = 0.8026785714285715
07/07/2021 07:07:08 - INFO - trainer -     f1 = 0.7031924217953387
07/07/2021 07:07:11 - INFO - trainer -   ***** Running evaluation *****
07/07/2021

Epoch:  73%|███████▎  | 22/30 [03:11<01:08,  8.62s/it]07/07/2021 07:08:08 - INFO - trainer -   ***** Running evaluation *****
07/07/2021 07:08:08 - INFO - trainer -     Num examples = 1120
07/07/2021 07:08:08 - INFO - trainer -     Batch size = 32
07/07/2021 07:08:09 - INFO - trainer -   ***** Eval results *****
07/07/2021 07:08:09 - INFO - trainer -     eval loss: 0.5286704891494343
07/07/2021 07:08:09 - INFO - trainer -     acc = 0.8008928571428572
07/07/2021 07:08:09 - INFO - trainer -     f1 = 0.7103597361556636
07/07/2021 07:08:12 - INFO - trainer -   ***** Running evaluation *****
07/07/2021 07:08:12 - INFO - trainer -     Num examples = 1120
07/07/2021 07:08:12 - INFO - trainer -     Batch size = 32
07/07/2021 07:08:13 - INFO - trainer -   ***** Eval results *****
07/07/2021 07:08:13 - INFO - trainer -     eval loss: 0.5869143947958946
07/07/2021 07:08:13 - INFO - trainer -     acc = 0.7883928571428571
07/07/2021 07:08:13 - INFO - trainer -     f1 = 0.6839134561333817
Epoch:  77

07/07/2021 07:09:11 - INFO - trainer -     acc = 0.7857142857142857
07/07/2021 07:09:11 - INFO - trainer -     f1 = 0.6820553407700896
07/07/2021 07:09:15 - INFO - trainer -   ***** Running evaluation *****
07/07/2021 07:09:15 - INFO - trainer -     Num examples = 1120
07/07/2021 07:09:15 - INFO - trainer -     Batch size = 32
07/07/2021 07:09:15 - INFO - trainer -   ***** Eval results *****
07/07/2021 07:09:15 - INFO - trainer -     eval loss: 0.5891395215477262
07/07/2021 07:09:15 - INFO - trainer -     acc = 0.8053571428571429
07/07/2021 07:09:15 - INFO - trainer -     f1 = 0.715551294462741
Epoch: 100%|██████████| 30/30 [04:22<00:00,  8.74s/it]
07/07/2021 07:09:18 - INFO - __main__ -     acc = 0.8133928571428571
07/07/2021 07:09:18 - INFO - __main__ -     f1 = 0.7244278215763696
